The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide.

In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

In [3]:
decay_chain_dir = parent_dir * "ExportedData\\all_beta_decay_chains\\"
decay_chain_files = readdir(decay_chain_dir)
all_decay_chains = [CSV.read(decay_chain_dir * file, DataFrame)
                    for file in decay_chain_files]
all_decay_chains = [rename!(df, :Column1 => :Parent) for df in all_decay_chains]

3192-element Vector{DataFrame}:
 4×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Ag                   173.989                         5.69  100Cd
   2 │ 100Cd                    70.8363                        5.69  100In
   3 │ 100In                     8.10795                       5.69  100Sn
   4 │ 100Sn                     1.70238                       5.69  100Sb
 3×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Cd                    70.8363                        5.69  100In
   2 │ 100In                     8.10795                       5.69  100Sn
   3 │ 100Sn  

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [4]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [5]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [6]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      /
                      -e_fold_times[1])


r (generic function with 1 method)

In [7]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / (-λ_0)

(-(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [8]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [9]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [10]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [11]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 1 method)

In [12]:
power_per_mole(e_folding_times, decay_energies, t) = sum([
    power_per_mole(r(e_folding_times[1:i], t), decay_energies[i])
    for i in 1:length(decay_energies)])

power_per_mole (generic function with 2 methods)

In [13]:
E_0, E_1, E_2, t = @variables E_0, E_1, E_2, t
expected_output = conversion_factor * sum([
    E_0 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / (-λ_0)
])

((-6475021031458043//67108864)*E_0*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [14]:
decay_energies = [E_0, E_1, E_2]
Test.@test Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) - expected_output) == 0
   Evaluated: 0 == 0

In [17]:
x = [power_per_mole(df[!,"e-Folding Time (seconds)"], df[!,"Average beta-decay energy"], 2)
for df in all_decay_chains];

It takes about .1 seconds to calculate the power per mole for the entire dataframe for a given time. 

In [27]:
exponents_for_time_array = -10:20/1000:10
collect(exponents_for_time_array)

1001-element Vector{Float64}:
 -10.0
  -9.98
  -9.96
  -9.94
  -9.92
  -9.9
  -9.88
  -9.86
  -9.84
  -9.82
  -9.8
  -9.78
  -9.76
   ⋮
   9.78
   9.8
   9.82
   9.84
   9.86
   9.88
   9.9
   9.92
   9.94
   9.96
   9.98
  10.0

In [31]:
time_array = [BigFloat(10)^x for x in exponents_for_time_array]

1001-element Vector{BigFloat}:
 1.000000000000000000000000000000000000000000000000000000000000000000000000000003e-10
 1.047128548050898505549645783823682271757193893433574137820153812424537637016191e-10
 1.0964781961431828604257317209264159331313714105011702911985913060102831306613e-10
 1.148153621496884066480156540139268838588675605873626387416861363160599339507005e-10
 1.202264434617413102533306847068551134853580025789314084263608408653746937282328e-10
 1.25892541179416618056939287182433126845996218444738646667433027351526766150798e-10
 1.318256738556404729589129642168660706457656431654852798000801526451019756623137e-10
 1.38038426460288664458362703761266589785155477230948719984049693714804016050564e-10
 1.445439770745927984903979184458196052506877843919927508765240153411912508988761e-10
 1.513561248436207171938059987874779303581263483786977823789873399913939914213657e-10
 1.584893192461110892181946192724010648088344291738932269688212001583469968929013e-10
 1.659586907437563078013118

In [55]:
power_time_series = [[power_per_mole(df[!,"e-Folding Time (seconds)"], 
                    df[!,"Average beta-decay energy"], time)
                    for time in time_array] for df in all_decay_chains]; 
zero_gen_isotopes = [df[1, "Parent"] for df in all_decay_chains]
power_time_series_dict = Dict([])
stop = length(power_time_series)
for i in 1:stop
        power_time_series_dict[zero_gen_isotopes[i]] = power_time_series[i]
end
power_time_series_dict

Dict{Any, Any} with 3192 entries:
  "113Sn" => BigFloat[-19.1026, -19.1026, -19.1026, -19.1026, -19.1026, -19.102…
  "243Cf" => BigFloat[-587251.0, -587251.0, -587251.0, -587251.0, -587251.0, -5…
  "168Eu" => BigFloat[-1.90269e+09, -1.90269e+09, -1.90269e+09, -1.90269e+09, -…
  "162Sm" => BigFloat[-1.4094e+08, -1.4094e+08, -1.4094e+08, -1.4094e+08, -1.40…
  "11N"   => BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0,…
  "57V"   => BigFloat[-1.08725e+09, -1.08725e+09, -1.08725e+09, -1.08725e+09, -…
  "208Po" => BigFloat[-4.16099, -4.16099, -4.16099, -4.16099, -4.16099, -4.1609…
  "174Ir" => BigFloat[-4.81695e+07, -4.81695e+07, -4.81695e+07, -4.81695e+07, -…
  "144Gd" => BigFloat[-1.41886e+06, -1.41886e+06, -1.41886e+06, -1.41886e+06, -…
  "80Sr"  => BigFloat[-59664.3, -59664.3, -59664.3, -59664.3, -59664.3, -59664.…
  "33Ar"  => BigFloat[-2.19965e+09, -2.19965e+09, -2.19965e+09, -2.19965e+09, -…
  "220Pa" => BigFloat[-4.48174e+14, -4.48197e+14, -4.48221e+14, -4.48246e+1

In [91]:
function plot_power_time_series(nuclide, time_array)
    power_time_series = Float64.(power_time_series_dict[nuclide]) * -1
    time_array = Float64.(time_array)
    plot(scatter(x = time_array, y = power_time_series, mode = "markers"), 
    Layout(xaxis_type = "log"))#, yaxis_type = "log"))
end
plot_power_time_series("207Bi", time_array)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, and xaxis"

In [92]:
values(power_time_series_dict)

ValueIterator for a Dict{Any, Any} with 3192 entries. Values:
  BigFloat[-19.1025568798946123668504474334058123901375513715858304473797140489…
  BigFloat[-587251.341703489316208504342641759751866235999819376470665779264549…
  BigFloat[-1.90269434649764424515051666861615020476685990715511260200109611593…
  BigFloat[-1.40940322004792226184771177973244039937010310959979527893717552811…
  BigFloat[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, …
  BigFloat[-1.08725391242455897152540011849423316945974926889205591659174234684…
  BigFloat[-4.16099003137274607803492289915462319490625870241081085318375720462…
  BigFloat[-4.81694771455657127912030668422098223663956049486817379362299572005…
  BigFloat[-1.41886230209772572032194310598118142012780999580357279804617260917…
  BigFloat[-59664.2943592945136360682011143552122466182434613439814563208638205…
  BigFloat[-2.19964664318058041102015665499128540050214981542593211172329692489…
  BigFloat[-4.481743692645840254761350799488506

In [104]:
power_matrix = Matrix{Union{Float64, BigFloat}}(undef, stop, length(time_array))
for row in 1:stop
    power_matrix[row, :] = power_time_series_dict[zero_gen_isotopes[row]]
end
power_df = DataFrame(power_matrix, string.(time_array))

,1.0e-10
,Union…
1,-3.155380343474517773182881336006113448165850785347324068054552557712315167057752e+06
2,-7.750282473052230410131589151789001926210682420581984528770889009429861047264477e+06
3,-6.771154260504771926121105292668354778399529497285304976772711638720867902521081e+07
4,-3.171157231156666882975197464330423674578587447320415567899638881508334689581838e+10
5,-1.705593933659425828895044885609053134811681637366715319113371576457134049742354e-18
6,-2.53692579603415792778394416593882461800050762529806888346674704412560119165307e+08
7,-509.6409195010121875533388457038160623571648639965189717384238721768647758216825
8,-7.417911677174506856344579947799272975701897108371049300218245589944594604179178e+09
9,-289378.2804519037072109144206399175771913850710015473516742436811413904833393907


In [106]:
power_df[!, "Parent Isotope"] = zero_gen_isotopes

3192-element Vector{InlineString}:
 "100Ag"
 "100Cd"
 "100In"
 "100Kr"
 "100Mo"
 "100Nb"
 "100Pd"
 "100Rb"
 "100Rh"
 "100Sn"
 "100Sr"
 "100Tc"
 "100Y"
 ⋮
 "99Pd"
 "99Rb"
 "99Rh"
 "99Sn"
 "99Sr"
 "99Tc"
 "99Y"
 "99Zr"
 "9B"
 "9C"
 "9He"
 "9Li"

In [ ]:
CSV.write(parent_dir * "ExportedData\\")